In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
from sklearn.preprocessing import LabelEncoder



df_books = pd.read_csv('C:/Users/vmaru/Desktop/Economics 2/Books.csv')

df_users = pd.read_csv('C:/Users/vmaru/Desktop/Economics 2/Users.csv')

df_rating = pd.read_csv('C:/Users/vmaru/Desktop/Economics 2/Ratings.csv')



C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\2181297592.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv('C:/Users/vmaru/Desktop/Economics 2/Books.csv')


In [2]:
df_books.isna().sum()



ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [3]:
df_books[df_books['Book-Author'].isna()]

df_books['Book-Author'][df_books['ISBN'] == '0751352497'] = 'Unknown'
df_books['Book-Author'][df_books['ISBN'] == '9627982032'] = 'Larissa Anne Downes'

C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\625118924.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_books['Book-Author'][df_books['ISBN'] == '0751352497'] = 'Unknown'
C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\625118924

In [4]:
df_books[df_books['Publisher'].isna()]

df_books['Publisher'][df_books['ISBN'] == '193169656X'] = 'Mundania Pr'

df_books['Publisher'][df_books['ISBN'] == '1931696993'] = 'Bantam'


C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\1435067118.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_books['Publisher'][df_books['ISBN'] == '193169656X'] = 'Mundania Pr'
C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\143506

In [5]:
user_rating = pd.merge(df_users,df_rating, on = 'User-ID')

df = pd.merge(df_books, user_rating, on = 'ISBN')

df.isna().sum()


ISBN                        0
Book-Title                  0
Book-Author                 0
Year-Of-Publication         0
Publisher                   0
Image-URL-S                 0
Image-URL-M                 0
Image-URL-L                 4
User-ID                     0
Location                    0
Age                    277835
Book-Rating                 0
dtype: int64

In [6]:

le = LabelEncoder()
df['Publisher_encoded'] = le.fit_transform(df['Publisher'])
df['Location_encoded'] = le.fit_transform(df['Location'])


In [7]:
kk = df['Year-Of-Publication'].unique()

df['Year-Of-Publication'] = pd.to_numeric(df['Year-Of-Publication'], errors='coerce')

df['Book-Author'][df['Year-Of-Publication'].isna()] = 'James Buckley'
df['Year-Of-Publication'][df['Year-Of-Publication'].isna()] = 2000

df = df[df['Book-Rating'] > 0]

C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\2002851963.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Book-Author'][df['Year-Of-Publication'].isna()] = 'James Buckley'
C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\20028519

In [8]:
df['Year-Of-Publication'] = df['Year-Of-Publication'].astype(int)

df['ISBN'] = df['ISBN'].str.upper()

In [9]:
import json

def year_scraping(isbn):
    url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data"

    response = requests.get(url)
    book_data = response.json()


    
    isbn_key = next(iter(book_data))
    year = book_data[isbn_key].get("publish_date", "Unknown")


    return year

In [10]:
df_susp = df[df['Year-Of-Publication'] <= 200]

df_susp

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Location,Age,Book-Rating,Publisher_encoded,Location_encoded
12969,3150000335,Kabale Und Liebe,Schiller,0,"Philipp Reclam, Jun Verlag GmbH",http://images.amazon.com/images/P/3150000335.0...,http://images.amazon.com/images/P/3150000335.0...,http://images.amazon.com/images/P/3150000335.0...,242,"neuffen, baden-wuerttemberg, germany",37.0,10,11527,13712
12973,3150000335,Kabale Und Liebe,Schiller,0,"Philipp Reclam, Jun Verlag GmbH",http://images.amazon.com/images/P/3150000335.0...,http://images.amazon.com/images/P/3150000335.0...,http://images.amazon.com/images/P/3150000335.0...,78761,"heide, schleswig-holstein, germany",34.0,7,11527,8496
12974,3150000335,Kabale Und Liebe,Schiller,0,"Philipp Reclam, Jun Verlag GmbH",http://images.amazon.com/images/P/3150000335.0...,http://images.amazon.com/images/P/3150000335.0...,http://images.amazon.com/images/P/3150000335.0...,107951,"tübingen, baden-württemberg, germany",NaN,7,11527,20306
13027,342311360X,Die Liebe in Den Zelten,Gabriel Garcia Marquez,0,Deutscher Taschenbuch Verlag (DTV),http://images.amazon.com/images/P/342311360X.0...,http://images.amazon.com/images/P/342311360X.0...,http://images.amazon.com/images/P/342311360X.0...,242,"neuffen, baden-wuerttemberg, germany",37.0,10,4016,13712
13028,342311360X,Die Liebe in Den Zelten,Gabriel Garcia Marquez,0,Deutscher Taschenbuch Verlag (DTV),http://images.amazon.com/images/P/342311360X.0...,http://images.amazon.com/images/P/342311360X.0...,http://images.amazon.com/images/P/342311360X.0...,162233,"zuerich, zuerich, switzerland",35.0,9,4016,22430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030159,0713715200,American Filmmakers Today,Dian Smith,0,Sterling*+ Publishing Company,http://images.amazon.com/images/P/0713715200.0...,http://images.amazon.com/images/P/0713715200.0...,http://images.amazon.com/images/P/0713715200.0...,243223,"budapest, n/a, hungary",27.0,5,14148,2873
1030212,0553026828,Dissertation,R M Koster,0,Bantam Doubleday Dell,http://images.amazon.com/images/P/0553026828.0...,http://images.amazon.com/images/P/0553026828.0...,http://images.amazon.com/images/P/0553026828.0...,243490,"fort collins, colorado, usa",NaN,8,1431,6793
1030235,0517209802,Christmas Memories With Recipes,Compilation,0,Random House,http://images.amazon.com/images/P/0517209802.0...,http://images.amazon.com/images/P/0517209802.0...,http://images.amazon.com/images/P/0517209802.0...,243700,"milwaukee, wisconsin, usa",NaN,5,12360,12658
1030322,0571204163,Headlong,Michael Frayn,0,Faber Faber Inc,http://images.amazon.com/images/P/0571204163.0...,http://images.amazon.com/images/P/0571204163.0...,http://images.amazon.com/images/P/0571204163.0...,243942,"baltimore, maryland, usa",NaN,8,5180,1312


In [12]:
import requests

unique_isbns = set(df_susp['ISBN'].dropna())

isbn_to_year = {}

for isbn in unique_isbns:
    try:
        isbn_to_year[isbn] = year_scraping(isbn)
    except StopIteration:
        continue  # Only makes sense inside a loop



In [13]:
df_susp['Year-Of-Publication'] = df_susp['ISBN'].map(isbn_to_year)

df_susp['Year-Of-Publication'][df_susp['Year-Of-Publication'].isna()] = 'Unknown'


C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\1808039286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_susp['Year-Of-Publication'] = df_susp['ISBN'].map(isbn_to_year)
C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\1808039286.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] 

In [14]:
import re

df_susp['Year-Of-Publication'] = df_susp['Year-Of-Publication'].apply(lambda x: re.findall(r'\b\d{4}\b', str(x)))


df_susp['Year-Of-Publication'] = df_susp['Year-Of-Publication'].apply(lambda x: int(x[0]) if isinstance(x, list) and x else 0)


C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\1506653776.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_susp['Year-Of-Publication'] = df_susp['Year-Of-Publication'].apply(lambda x: re.findall(r'\b\d{4}\b', str(x)))
C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\1506653776.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_susp['Year-Of-Publication'] = df_susp['Year-Of-Publication'].apply(lambda x: int(x[0]) if isinstance(x, list) and x else 0)


In [ ]:
df_susp_1 = df[df['Year-Of-Publication'] >= 2024]

unique_isbns_1 = set(df_susp_1['ISBN'].dropna())

isbn_to_year_1 = {}

for isbn in unique_isbns_1:
    try:
        isbn_to_year_1[isbn] = year_scraping(isbn)
    except StopIteration:
        continue  # Only makes sense inside a loop

df_susp_1['Year-Of-Publication'] = df_susp_1['ISBN'].map(isbn_to_year_1)


df_susp_1['Year-Of-Publication'] = df_susp_1['Year-Of-Publication'].apply(lambda x: re.findall(r'\b\d{4}\b', str(x)))

df_susp_1['Year-Of-Publication'] = df_susp_1['Year-Of-Publication'].apply(lambda x: int(x[0]) if isinstance(x, list) and x else 0)


df_susp_0 = pd.concat([df_susp, df_susp_1])

set_isbn = set(df_susp_0['ISBN'])



C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\2706216308.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_susp_1['Year-Of-Publication'] = df_susp_1['ISBN'].map(isbn_to_year_1)
C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\2706216308.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_susp_1['Year-Of-Publication'] = df_susp_1['Year-Of-Publication'].apply(lambda x: re.findall(r'\b\d{4}\b', str(x)))
C:\Users\vmaru\AppData\Local\Temp\ipykernel_15772\2706216308.py:18: SettingWithCopyWarning: 


In [32]:
new = df

df_susp_0 = df_susp_0.drop_duplicates(subset=['ISBN'])


In [33]:
new = new.merge(df_susp_0[['ISBN', 'Year-Of-Publication']], how = 'left', on = 'ISBN')

new['Year-Of-Publication_x'] = new['Year-Of-Publication_y'].combine_first(new['Year-Of-Publication_x'])

new.drop(columns = ['Year-Of-Publication_y'], inplace= True)



In [34]:
new.to_csv('clean_df.csv', index = False)

In [250]:
isbn = '0523410530'

url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data"

response = requests.get(url)
book_data = response.json()

book_data

{'ISBN:0523410530': {'url': 'https://openlibrary.org/books/OL10439181M/Hypoglycemia',
  'key': '/books/OL10439181M',
  'title': 'Hypoglycemia',
  'subtitle': "The Disease Your Doctor Won't Treat",
  'authors': [{'url': 'https://openlibrary.org/authors/OL225828A/Jeraldine_Saunders',
    'name': 'Jeraldine Saunders'}],
  'identifiers': {'librarything': ['3343946'],
   'goodreads': ['7149574'],
   'isbn_10': ['0523410530'],
   'isbn_13': ['9780523410531'],
   'oclc': ['7025146'],
   'openlibrary': ['OL10439181M']},
  'publishers': [{'name': 'Pinnacle Books, Inc'}],
  'subjects': [{'name': 'Hypoglycemia',
    'url': 'https://openlibrary.org/subjects/hypoglycemia'},
   {'name': 'Recipes', 'url': 'https://openlibrary.org/subjects/recipes'},
   {'name': 'Diet therapy',
    'url': 'https://openlibrary.org/subjects/diet_therapy'}]}}

In [251]:
# Normalizing numerical data
scaler = StandardScaler()
numerical_features = scaler.fit_transform(df[['Book-Rating', 'Year-Of-Publication']])


ValueError: setting an array element with a sequence.

In [86]:
first_users = list(df['User-ID'][df["Book-Title"].str.contains("The Lord of the Rings")]) #& (df['Book-Rating'] > 6)

first_book = df[df['User-ID'].isin(first_users)]




In [87]:
from scipy.sparse import lil_matrix


user_filt = first_book['User-ID'].unique()

book_filt = first_book['ISBN'].unique()

n_book = len(book_filt)

n_user = len(user_filt)

matrix = lil_matrix((n_user, n_book), dtype=float)

# Create user and book index mappings
user_to_index = {user: idx for idx, user in enumerate(user_filt)}
book_to_index = {book: idx for idx, book in enumerate(book_filt)}


# Populate the sparse matrix
for _, row in first_book.iterrows():
    user_idx = user_to_index[row['User-ID']]
    book_idx = book_to_index[row['ISBN']]
    matrix[user_idx, book_idx] = row['Book-Rating']

from scipy.sparse import csr_matrix

rating_matrix = matrix.tocsr()

from sklearn.metrics.pairwise import cosine_similarity

# Transpose to get items (books) as rows
item_similarity = cosine_similarity(rating_matrix.T)

item_similarity

array([[1.        , 0.        , 0.78935222, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.78935222, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]], shape=(30026, 30026))

In [88]:
index_to_book = {idx: book for book, idx in book_to_index.items()}

def recommend_books_based_on_book(book_id, top_n=5):
    if book_id not in book_to_index:
        return pd.DataFrame(columns=["Book-Title", "ISBN", "Similarity-Score"])

    # Get the index of the input book
    book_idx = book_to_index[book_id]
    
    # Get similarity scores for this book
    similar_books = list(enumerate(item_similarity[book_idx]))
    
    # Sort by similarity scores (highest first), excluding the input book itself
    similar_books = sorted(similar_books, key=lambda x: x[1], reverse=True)
    
    # Select top N similar books (excluding the input book itself)
    recommended_books = [
        (index_to_book[idx], round(float(score), 4))
        for idx, score in similar_books[1:top_n + 1]
    ]
    
    # Create a DataFrame for the recommendations
    recommendations_df = pd.DataFrame(
        {
            "Book-Title": [
                first_book.loc[first_book["ISBN"] == isbn, "Book-Title"].values[0]
                for isbn, _ in recommended_books
            ],
            "Similarity-Score": [score for _, score in recommended_books],
        }
    )
    return recommendations_df

book_id = '0618260250'
recommendations = recommend_books_based_on_book(book_id, top_n=5)
print(recommendations)


                                          Book-Title  Similarity-Score
0  The Devil in the White City : Murder, Magic, a...            0.3535
1        Confessions of an Ugly Stepsister : A Novel            0.3476
2      Shock Wave (Dirk Pitt Adventures (Paperback))            0.3446
3                                The Dominant Blonde            0.3446
4             The Silver Chair (full color) (Narnia)            0.3446
